**Jonathan Rossi**  
*April 21, 2016*  
Python

---

# Single-Variable Linear Regression Algorithm #

This is an implementation of a single-variable linear regression algorithm. We use gradient descent to find the values
of $\theta_0$ and $\theta_1$—the intercept and slope (x-coefficient), respectively—that minimize the sum of squared residuals of the
regression line, which gives us a "line of best fit." This code is based on concepts used in the following places:

* https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/
* http://www.bogotobogo.com/python/python_numpy_batch_gradient_descent_algorithm.php
* https://www.coursera.org/learn/machine-learning/lecture/rkTp3/cost-function

## Sections ##

* Gradient Descent Primer
* Implementation
* Implementation Walkthrough
* Example Dataset and Results

## Gradient Descent Primer ##

The object of linear regression is to find a line that best fits a scatter plot of data points. A common definition of "line of best fit" is that the line minimizes the sum of the squared residuals (i.e., minimizes the sum of the distances between each point and the line). We define a "cost function" $J(\theta_0,\,\theta_1)$ that equals exactly this sum:

$$J(\theta_0,\,\theta_1) \,\, = \,\, \sum_{i=1}^{m} ((\theta_0 + \theta_1x_i) - y_i)^2$$

where $\theta_0 + \theta_1x_i$ is the equation for our regression line. $\theta_0$ represents the intercept of the regression line and $\theta_1$ represents the slope. Our goal is to find $\theta_0$ and $\theta_1$ such that $J(\theta_0,\,\theta_1)$ is minimized. In practice, we will minimize the following:

$$\underset{\theta_0,\,\theta_1}{\text{minimize}}\,\frac{1}{2m}\sum_{i=1}^{m} (\theta_0 + \theta_1x_i - y_i)^2$$

We adjust the sum by the term $\frac{1}{2m}$ by convention. Dividing by $m$ gives us the average squared residual, and the factor of $\frac{1}{2}$ gives us a cleaner expression when we take the derivative later on for gradient descent (the multiple of $2$ from the exponent cancels with the $\frac{1}{2}$).

### Gradient Descent ###

There are many ways to find $\theta_0$ and $\theta_1$ such that $J(\theta_0,\,\theta_1)$ is minimized. In fact, with a linear regression, we can compute the optimal values directly. However, part of the objective of this exercise was to use gradient descent, and so that is what we will do.

Gradient Descent is a widely used optimization algorithm. Applying it to our cost function, $J(\theta_0,\,\theta_1)$, we are going to follow two steps:

* Initialize $\theta_0$ and $\theta_1$ to be some values. There are smart ways to choose the values to initialize to but we will simply initialize to $0$ and that will serve our purposes.
* Repeatedly update the values for $\theta_0$ and $\theta_1$ until our algorithm converges on a minimum for $J(\theta_0,\,\theta_1)$.

Our update step will look like this:

$$\theta_j \, := \,\theta_j - \alpha\frac{\partial}{\partial\theta_j}J(\theta_0,\,\theta_1)\,\,\,\,\,\,\text{for $j$ = $0, 1$}$$

where $\alpha$ is the learning rate of the algorithm, which we can change to fine tune how the algorithm converges. It is important to note that in order for the algorithm to work properly, we need to update $\theta_0$ and $\theta_1$ simultaneously. We can accomplish that by doing the following:

$$temp_0 := \,\theta_0 - \alpha\frac{\partial}{\partial\theta_0}J(\theta_0,\,\theta_1)$$
$$temp_1 := \,\theta_1 - \alpha\frac{\partial}{\partial\theta_1}J(\theta_0,\,\theta_1)$$

and only then updating $\theta_0$ and $\theta_1$:

$$\theta_0 := temp_0$$
$$\theta_1 := temp_1$$

The following video gives some intuition behind why adjusting $\theta_0$ and $\theta_1$ by their respective partial derivatives of the cost function leads us to a minimum: https://www.youtube.com/watch?v=Fn8qXpIcdnI. In short, the slope of the cost function (i.e., the derivative) with respect to $\theta_j \,$ (for $j$ = $0,1$) is negative if we are too far to the left of the cost function's minimum (i.e., $\theta_j$ is too small). In this case, we subtract a negative, which makes $\theta_j$ bigger. If we are too far to the right of the minimum, the derivative of the cost function with respect to $\theta_j$ is positive. This time, we subtract a positive, which makes $\theta_j$ smaller. In this way, we move closer and closer to the optimal value of $\theta_j$. Furthermore, as we get closer to the minumum, the derivative of the cost function gets smaller, and so we adjust $\theta_j$ by a smaller and smaller amount each time. This allows us to converge on an optimal value.

Given our cost function $J(\theta_0,\,\theta_1)$, we take the partial derivatives with respect to $\theta_0$ and $\theta_1$:

$$\frac{\partial}{\partial\theta_j}\frac{1}{2m}\sum_{i=1}^{m} (\theta_0 + \theta_1x_i - y_i)^2\,\,\,\,\,\,\text{for $j$ = $0, 1$}$$

which evaluate to:

$$\frac{\partial}{\partial\theta_0}J(\theta_0,\,\theta_1) = \frac{1}{m}\sum_{i=1}^{m} (\theta_0 + \theta_1x_i - y_i)$$
$$\frac{\partial}{\partial\theta_1}J(\theta_0,\,\theta_1) = \frac{1}{m}\sum_{i=1}^{m} (\theta_0 + \theta_1x_i - y_i)\,x_i$$

So, our update step, using $J(\theta_0,\,\theta_1)$ as our cost function, looks like:

$$\theta_0 \, := \,\theta_0 - \alpha\frac{1}{m}\sum_{i=1}^{m} (\theta_0 + \theta_1x_i - y_i)$$
$$\theta_1 \, := \,\theta_1 - \alpha\frac{1}{m}\sum_{i=1}^{m} (\theta_0 + \theta_1x_i - y_i)\,x_i$$

## Implementation ##

In [32]:
import numpy as np
import pandas as pd
from scipy import stats

In [33]:
# Define the Single-Variable Linear Regression model class.
class SLRegression(object):
    def __init__(self, learnrate = .01, tolerance = .000000001, max_iter = 10000):
        # learnrate (float): the learning rate for the regression.
        # tolerance (float): our threshold for defining "convergence."
        # max_iter (int): the maximum number of iterations we will allow.

        self.learnrate = learnrate
        self.tolerance = tolerance
        self.max_iter = max_iter

    def change_learnrate(self, new_learnrate):
        # new_learnrate (float): the new learning rate.
        self.learnrate = new_learnrate

    def change_tolerance(self, new_tolerance):
        # new_tolerance (float): the new tolerance.
        self.tolerance = new_tolerance

    def change_max_iter(self, new_max_iter):
        # new_max_iter (int): thenew maximum number of iterations.
        self.max_iter = new_max_iter

    # Define fit function.
    def fit(self, data):
        # data (array-like, shape = [m_observations, 2_columns]): the training data.

        converged = False
        m = data.shape[0]
            # converged (bool): whether algorithm has converged.
            # m (int): the number of samples.

        # Initialize other class variables.
        self.iter_ = 0
        self.theta0_ = 0
        self.theta1_ = 0

        # Compute the "cost" function J.
        J = (1.0/(2.0*m)) * sum([(self.theta0_ + self.theta1_*data[i][1] - data[i][0])**2 for i in range(m)])

        # Recursively update theta0 and theta1.
        while not converged:
            self.iter_ += 1

            # Calculate the partial derivatives of J with respect to theta0 and theta1.
            pdtheta0 = (1.0/m) * sum([(self.theta0_ + self.theta1_*data[i][1] - data[i][0]) for i in range(m)])
            pdtheta1 = (1.0/m) * sum([(self.theta0_ + self.theta1_*data[i][1] - data[i][0]) * data[i][1] for i in range(m)])

            # Subtract the learnrate * partial derivative from theta0 and theta1.
            temp0 = self.theta0_ - (self.learnrate * pdtheta0)
            temp1 = self.theta1_ - (self.learnrate * pdtheta1)

            # Update theta0 and theta1.
            self.theta0_ = temp0
            self.theta1_ = temp1

            # Compute the updated cost function, given new theta0 and theta1.
            new_J = (1.0/(2.0*m)) * sum([(self.theta0_ + self.theta1_*data[i][1] - data[i][0])**2 for i in range(m)])

            # Test for convergence.
            if abs(J - new_J) <= self.tolerance:
                converged = True
                print(('Model converged after %s iterations!') % (self.iter_))

            # Set old cost equal to new cost.
            J = new_J

            # Test whether we have hit max_iter.
            if self.iter_ == self.max_iter:
                converged = True
                print('Maximum iterations have been reached!')

        return self
    
    # Define a "point forecast" function.
    def point_forecast(self, x):
        return self.theta0_ + self.theta1_ * x

## Implementation Walkthrough ##

This is a detailed, step-by-step explanation of the Python implementation of the algorithm.

### Define the model class. ###

* **Define the initializer.**
    
    * A note about `tolerance`: our "best fit" line is considered converged, when the difference between the cost function with the old values of `theta0` and `theta1` and updated cost function is less than or equal to the `tolerance`.
    

* **Define helper functions**, `change_learn_rate()`, `change_tolerance()`, and `change_max_iter()`, that allow us to easily change the initial parameters of the regression model.


* **Define a 'fit' function** that uses gradient descent to fit a regression line to our data (i.e., finds optimal `theta0` and `theta1`).

    * `data` has the shape `[m_observations, 2_columns]`, where the 2 columns are the dependent-variable values in the first column, and the independent-variable values in the second column.
    
    * Initialize local variables `converged` and `m`. We will continue to update `theta0` and `theta1` until `converged` is `True`, and `m` is the number of samples we pass to the function.
        * `data.shape` returns `(num_rows, num_cols)`, so `data.shape[0] = num_rows` and `data.shape[1] = num_cols`.
    * Initialize additional class variables, `self.iter_`, `self.theta0_`, and `self.theta1_`. By convention, we use `_` to denote any attributes of the SLRegression class that are not part of the class initialization.
        * Initialize the number of iterations, `self.iter_`, to `0` and track it as we go.
        * Initialize `self.theta0_` and `self.theta1_` to `0`. The values we use for initialization can affect how the algorithm converges, but initializing to `0` will serve our purposes.
        
    * Compute the "cost" function `J(theta0, theta1)`, which computes the sum of the squared residuals, given `theta0` and `theta1`. This is the sum we want to minimize in order to find optimal `theta1` and `theta0`. Note, the square brackets in the `sum` expression ensure that we return a `list` instead of a `generator` (either way, the result will be a single number).
    * Recursively update `theta0` and `theta1` until `converged == True` or `self.iter_ == self.max_iter`.
        * Increase `self.iter_`.
        * Calculate the partial derivatives, `pdtheta0` and `pdtheta1`, of J with respect to `theta0` and `theta1`. These are the "differences" by which we will adjust our current `theta0` and `theta1`.
        * Subtract the `learnrate` multiplied by the partial derivatives from `theta0` and `theta1`.
        * Update `theta0` and `theta1`.
        * Compute the updated cost function, given new `theta0` and `theta1`.
        * Test for convergence.
        * Set old cost equal to new cost.
        * Test whether we have hit max_iter.
        
    
    
* **Define a 'point forecast' function.**
    
    * Given feature value `x`, `point_forecast()` returns the regression's predicted value for `y`.

## Example Dataset and Results ##

In [34]:
%matplotlib inline
import matplotlib.pyplot as plt

In [35]:
# Read in the data (temperature (column 1) versus sales (column 2)).
data = np.squeeze(np.array(pd.read_csv('sales_normalized.csv')))

# Create a regression model with the default learning rate, tolerance, and maximum number of iterations.
slregression = SLRegression()

# Call the fit function and pass in our data.
slregression.fit(data)

# Print out the results.
print(('After %s iterations, the model converged on Theta0 = %s and Theta1 = %s.') % (slregression.iter_, slregression.theta0_, slregression.theta1_))

# Compare our model to scipy linregress model.
slope, intercept, r_value, p_value, slope_std_error = stats.linregress(data[:,1], data[:,0])
print(('Scipy linear regression gives intercept: %s and slope = %s.') % (intercept, slope))

# Test the model with a point forecast.
print(('As an example, our algorithm gives y = %s, given x = .87.') % (slregression.point_forecast(.87))) # Should be about .83.
print('The true y-value for x = .87 is about .8368.')

Model converged after 8034 iterations!
After 8034 iterations, the model converged on Theta0 = 0.0731944487608 and Theta1 = 0.870240770913.
Scipy linear regression gives intercept: 0.0706191941281 and slope = 0.874682774148.
As an example, our algorithm gives y = 0.830303919455, given x = .87.
The true y-value for x = .87 is about .8368.
